# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!pip install nltk
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sadiatabassum/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sadiatabassum/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sadiatabassum/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import re
import pickle
from sqlalchemy import create_engine, inspect
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    recall_score,
    f1_score,
    precision_score,
    roc_auc_score,
    make_scorer,
    confusion_matrix,
    roc_curve,
    auc,
    matthews_corrcoef,
)
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
# from sklearn.linear_model import LogisticRegression


In [4]:
# # import libraries
# import pandas as pd
# import numpy as np
# from sqlalchemy import create_engine
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.base import BaseEstimator, TransformerMixin
# from sqlalchemy import create_engine, inspect
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.linear_model import LogisticRegression
# import sys
# import os
# import re
# import pickle

# from sklearn.metrics import accuracy_score, classification_report, recall_score, f1_score, precision_score, roc_auc_score, make_scorer
# from sklearn.metrics import confusion_matrix, make_scorer, roc_curve, auc, matthews_corrcoef


In [5]:
# load data from database
database_filepath = "../data/DisasterResponse.db"
engine = create_engine('sqlite:///' + database_filepath)
table_name = os.path.basename(database_filepath).replace(".db","")+ "_table"
df = pd.read_sql_table(table_name,engine)

In [6]:
df = pd.read_sql_table(table_name,engine)

In [7]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [8]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)
    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()
    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    
    return clean_tokens

In [10]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

KeyboardInterrupt: 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def eval_result(y_test, y_pred):
    y_pred_pd = pd.DataFrame(y_pred, columns=y_test.columns)
    res=[]
    col_names = y_test.columns
    for col in col_names:
        accuracy = accuracy_score(y_test[col], y_pred_pd[col])
        precision = precision_score(y_test[col], y_pred_pd[col],zero_division=1)
        recall = recall_score(y_test[col], y_pred_pd[col],zero_division=1)
        gmean = 2 * (recall * accuracy) / (recall + accuracy)
        f1score = f1_score(y_test[col], y_pred_pd[col],zero_division=1)
        res.append([accuracy, precision, recall, gmean, f1score])
        # print(f"{col}, Accuracy: {accuracy:.2f}, Recall: {recall:.2f}, G-Mean: {gmean:.2f}, F1 score: {f1score:.2f}")
    res = np.array(res)
    res_df = pd.DataFrame(data = res, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'G-Mean','F1'])
    return res_df

In [ ]:
# y_prediction_train = pipeline_model1.predict(X_train)
y_pred = pipeline.predict(X_test)


In [ ]:
eval_outcome_0 = eval_result(y_test,y_pred)
print(eval_outcome_0)

> Even though the accuracy is high for most categories, F1 scores and G-Mean are low for most categories. This is possibly due to class imbalance issue in the data, as many categories have very low percentage of label 1.

### 6. Improve your model
Use grid search to find better parameters. 
We used G-Mean as the performance metric here as G-Mean is suitable metric for class imbalanced data [2][3].

In [ ]:
# Define performance metric for use in grid search scoring object
def grid_eval_metric(y_true, y_pred):
    """Calculate median gmean score for all of the output classifiers
    
    Args:
    y_true: array. Array containing actual labels.
    y_pred: array. Array containing predicted labels.
        
    Returns:
    score: float. Median gmean score for all of the output classifiers
    """
    gmean_list = []
    
    for i in range(np.shape(y_pred)[1]):
        accuracy = accuracy_score(np.array(y_true)[:, i], y_pred[:, i])
        recall = recall_score(np.array(y_true)[:, i], y_pred[:, i],zero_division=1)
        gmean = 2 * (recall * accuracy) / (recall + accuracy)
        gmean_list.append(gmean)
       
        
    score = np.median(gmean_list)
    return score

In [ ]:
# grid_eval_metric(y_test,y_pred)

In [ ]:
parameters = {
        'clf__estimator__n_estimators': [10, 20], 
        'clf__estimator__min_samples_split': [2, 5, 10] 
    }
scorer = make_scorer(grid_eval_metric)
cv = GridSearchCV(pipeline, param_grid=parameters, scoring = scorer, verbose = 10)

In [ ]:
tuned_model = cv.fit(X_train, y_train)

In [ ]:
tuned_model.cv_results_

In [ ]:
tuned_model.best_params_

In [ ]:
# cv.get_params().keys()

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
tuned_y_pred = tuned_model.predict(X_test)

In [ ]:
tuned_eval_outcome = eval_result(y_test,tuned_y_pred)
print(tuned_eval_outcome)

In [ ]:
tuned_eval_outcome.describe()

In [ ]:
eval_outcome_0.describe()

> The results demonstrate improved model performance following the tuning process. Notably, the average G-Mean for the tuned model (0.285999) exceeded the G-Mean of the untuned model (0.261217). Similarly, the average F1 score for the tuned model (0.279163) surpassed the F1 score of the untuned model (0.257834).

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

To improve the model further, we added a custom transformer in the pipeline that extracts the starting verb of a sentence. We also used a different classifier `AdaBoostClassifier` instead of `RandomForestClassifier`. 
AdaBoost is a boosting algorithm and is widely used to process imbalanced data [[1](https://www.mdpi.com/1424-8220/19/6/1476)]. 

In [ ]:
pipeline2 =Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),
            ('starting_verb_transformer', StartingVerbExtractor())
        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline2.fit(X_train, y_train)


In [ ]:
y_pred2 = pipeline2.predict(X_test)

In [ ]:
eval_result(y_test,y_pred2)

### Tuning

In [ ]:
parameters_grid = {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [10, 20, 40]}

In [ ]:
scorer2 = make_scorer(grid_eval_metric)
cv2 = GridSearchCV(pipeline2, param_grid=parameters_grid, scoring = scorer2, verbose = 10)

In [ ]:
tuned_model2 = cv2.fit(X_train, y_train)

In [ ]:
tuned_model2.cv_results_

In [ ]:
tuned_model2.best_params_

### Test improved model

In [ ]:
tuned_y_pred2 = tuned_model2.predict(X_test)

In [ ]:
tuned_eval_outcome2 = eval_result(y_test,tuned_y_pred2)
print(tuned_eval_outcome2)

In [ ]:
tuned_eval_outcome2.describe()

In [ ]:
tuned_eval_outcome.describe()

### 9. Export your model as a pickle file

In [ ]:
# model_final = pickle.dumps(filename)
filename = 'classifier.pkl'
with open(filename, 'wb') as file:
        pickle.dump(tuned_model2, file)

### References

[1] Li, K., Zhou, G., Zhai, J., Li, F. and Shao, M., 2019. Improved PSO_AdaBoost ensemble algorithm for imbalanced data. Sensors, 19(6), p.1476.

[2] Guo, H., Liu, H., Wu, C., Zhi, W., Xiao, Y. and She, W., 2016. Logistic discrimination based on G-mean and F-measure for imbalanced problem. Journal of Intelligent & Fuzzy Systems, 31(3), pp.1155-1166.

[3] Powers, D.M., 2020. Evaluation: from precision, recall and F-measure to ROC, informedness, markedness and correlation. arXiv preprint arXiv:2010.16061.